In [1]:
import sys
import os.path as osp
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')

import torch

from tools.logger import setup_logger
from tools.utils import deploy_macro, print_config
from src.factory.config_factory import _C as cfg
from src.factory.loader_factory import LoaderFactory
from src.factory.graph_factory import GraphFactory
from src.solver.solver import Solver
from src.factory.engine_factory import EngineFactory

In [2]:
logger = setup_logger(".")
deploy_macro(cfg)
cfg.merge_from_file('/media/allen/mass/deep-learning-works/configs/object.yml')
loader = LoaderFactory.produce(cfg)
graph = GraphFactory.produce(cfg)
solver = Solver(cfg, graph.model.named_parameters())

2020-03-05 17:05:23,615 logger INFO: Using GPU: 
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
2020-03-05 17:05:24,215 logger INFO: => COCO VAL is loaded
2020-03-05 17:05:24,216 logger INFO:   Dataset statistics:
2020-03-05 17:05:24,216 logger INFO:   -------------------
2020-03-05 17:05:24,217 logger INFO:   subset   | # images
2020-03-05 17:05:24,218 logger INFO:   -------------------
2020-03-05 17:05:24,218 logger INFO:   val      |     5000
2020-03-05 17:05:24,219 logger INFO:   -------------------
2020-03-05 17:05:24,300 logger INFO: Trainable parameters: 1.39M
2020-03-05 17:05:24,338 logger INFO: Cosine lr policy is used


In [7]:
from apex.fp16_utils import *
from apex import amp, optimizers

In [3]:
graph.use_multigpu()
p = next(iter(graph.model.parameters()))
p.is_cuda

2020-03-05 17:05:29,471 logger INFO: Using GPU: 
2020-03-05 17:05:29,519 logger INFO: GPU is not used


False

In [8]:
graph.model, solver.opt = amp.initialize(graph.model, solver.opt,
                                  opt_level='O3',
                                  keep_batchnorm_fp32=True
                                  )

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : False
loss_scale             : 1.0


In [9]:
graph.model = torch.nn.DataParallel(graph.model)

In [11]:
p = next(iter(graph.model.parameters()))

In [15]:
batch = next(iter(loader['val']))

In [16]:
for key in batch:
    batch[key] = batch[key].cuda()

In [17]:
outputs = graph.model(batch['inp'])

ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/apex/amp/_initialize.py", line 197, in new_fwd
    **applier(kwargs, input_caster))
  File "/media/allen/mass/deep-learning-works/src/graph/centernet_object_detection.py", line 14, in forward
    ps = self.backbone(x)
  File "/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/media/allen/mass/deep-learning-works/src/model/backbone/shuffle

In [ ]:
for batch in loader['train']:
    self._train_iter_start()
    for key in batch:
        batch[key] = batch[key].cuda()
    images = batch['inp']            
    outputs = self.graph.model(images)
    self.loss, self.losses = self.graph.loss_head(outputs, batch)
    self.loss.backward()
    self._train_iter_end()
    self.loss = self.tensor_to_scalar(self.loss)
    self.losses = self.tensor_to_scalar(self.losses)
